In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
# Importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Loading the movie data
!ls dataset/

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


In [5]:
# Reading the data

df_movies = pd.read_csv('dataset/movies.csv')
df_links = pd.read_csv('dataset/links.csv')
df_ratings = pd.read_csv('dataset/ratings.csv')
df_genome_tags = pd.read_csv('dataset/genome-tags.csv')
df_genome_scores = pd.read_csv('dataset/genome-scores.csv')

#merging scores and tags.
df_movie_tags_in_text = pd.merge(df_genome_scores, df_genome_tags, on='tagId')[['movieId', 'tag', 'relevance']]

# keep tags with relevnce higher than 0.3
df_movie_tags = df_genome_scores[df_genome_scores.relevance > 0.3][['movieId', 'tagId']]